In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import dump
import csv
from surprise import accuracy
from pprint import pprint

In [2]:
csv_path = os.path.join("../data/csv/reviews_cleaned_reduced_500.csv")
TextFileReader = pd.read_csv(csv_path, chunksize=1000)  # the number of rows per chunk

dfList = []
for df in TextFileReader:
    dfList.append(df)

df = pd.concat(dfList,sort=False)


In [3]:
# load Team Beer Ratings
csv_path = os.path.join("../data/csv/team_beers.csv")
team_beers_df = pd.read_csv(csv_path)

FileNotFoundError: [Errno 2] File ../data/csv/team_beers.csv does not exist: '../data/csv/team_beers.csv'

In [ ]:
# load beers
csv_path = os.path.join("../data/csv/beers.csv")
beers_df = pd.read_csv(csv_path)
beers_df = beers_df.rename(columns={'id': 'beer_id'})

In [5]:
# Add Team Beers to the reviews
df = pd.concat([df,team_beers_df])

In [6]:
# Lets combine the dataframe
merge_df = pd.merge(df,
                 beers_df[['beer_id', 'name', 'style', 'brewery_id']],
                 on='beer_id')

In [7]:
# sample random trainset and testset method using SVD
# test set is made of 25% of the ratings. 
reader=Reader(rating_scale=(0,5))
data = Dataset.load_from_df(merge_df[['username', 'beer_id', 'score']], reader)

trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.3876
MAE:  0.2785


0.2784633334563133

In [8]:
# Lets serialize and save this prediction algorithm
# Dump algorithm and reload it
dumpfile = os.path.join('../data/dump/dump_svd_dump_file')
dump.dump(dumpfile, predictions, algo)